## Introduction
- adjacency matrix
- sparse matrices in PyTorch
- message passing example

In [1]:
import torch

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/6n-graph2.svg/800px-6n-graph2.svg.png" width="200" height="200"/>
</div>


In [2]:
# NxN adjacency matrix
adj = torch.tensor([
    [2, 1, 0, 0, 1, 0],
    [1, 0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1, 1],
    [1, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0],
], dtype=torch.float)

In [3]:
# NxF node feature matrix: each node has 3 associated features
fea = torch.tensor([
    [0.10, 0.20, 0.10],
    [0.50, 0.0, 0.25],
    [0.50, 0.0, 0.25],
    [0.25, 0.25, 0.50],
    [0.30, 0.45, 0.40],
    [0.90, 0.60, 0.30],
])

In [4]:
# message passing is a matrix multiplication plus a matrix addition:
# new features computed from the node's previous features and its neighbors' previous features
fea + torch.mm(adj, fea)

tensor([[1.1000, 1.0500, 0.9500],
        [1.4000, 0.6500, 1.0000],
        [1.2500, 0.2500, 1.0000],
        [1.9500, 1.3000, 1.4500],
        [1.1500, 0.9000, 1.2500],
        [1.1500, 0.8500, 0.8000]])

In [5]:
# turn to sparse matrix format
# real-world graphs are very sparse: the number of their edges is a small fraction of the possible NxN edges
# so we can sparse a large amount of computer memory storing the adjacency matrix this way
adj.to_sparse()

tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5],
                       [0, 1, 4, 0, 2, 4, 1, 3, 2, 4, 5, 0, 1, 3, 3]]),
       values=tensor([2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1.]),
       size=(6, 6), nnz=15, layout=torch.sparse_coo)

In [6]:
# torch supports sparse-dense matrix multiplication
# we can see the same result as before
fea + torch.sparse.mm(adj.to_sparse(), fea)

tensor([[1.1000, 1.0500, 0.9500],
        [1.4000, 0.6500, 1.0000],
        [1.2500, 0.2500, 1.0000],
        [1.9500, 1.3000, 1.4500],
        [1.1500, 0.9000, 1.2500],
        [1.1500, 0.8500, 0.8000]])

In [7]:
# networkx is a python package to create, manipulate and visualize graphs
import networkx as nx

N = 20000
p = 0.02

# let's generate the sparse adjacency matrix of a larger graph (10000 nodes)
adj = nx.to_scipy_sparse_array(
    nx.fast_gnp_random_graph(N, p, seed=42), # generates an Erdős-Rényi random graph
    format='coo',
)

# we convert it to pytorch sparse matrix
adj = torch.sparse_coo_tensor(
    torch.tensor([adj.row, adj.col], dtype=torch.long),
    torch.tensor(adj.data, dtype=torch.float),
    size=torch.Size(adj.shape)
)


<ipython-input-7-b2bbf11fb98b>:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  torch.tensor([adj.row, adj.col], dtype=torch.long),


In [8]:
import time

fea = torch.rand((N, 3))

adj_cuda = adj.cuda()
fea_cuda = fea.cuda()

adj_dense = adj.to_dense()
adj_cuda_dense = adj.cuda().to_dense()

start = time.time()
torch.mm(adj_dense, fea)
stop = time.time()
print(f"dense-dense multiplication: {stop - start}")

start = time.time()
torch.mm(adj_cuda_dense, fea_cuda)
stop = time.time()
print(f"dense-dense multiplication on gpu: {stop - start}")

start = time.time()
torch.sparse.mm(adj, fea)
stop = time.time()
print(f"sparse-dense multiplication: {stop - start}")

start = time.time()
torch.mm(adj_cuda, fea_cuda)
stop = time.time()
print(f"sparse-dense multiplication on gpu: {stop - start}")


dense-dense multiplication: 0.2949855327606201
dense-dense multiplication on gpu: 0.018737077713012695
sparse-dense multiplication: 0.1868126392364502
sparse-dense multiplication on gpu: 0.0635378360748291


In [9]:
# conclusion: dense-dense matrix multiplication not only consumes high amount of memory,
# it is also considerably slower if most entries of the matrix are zero

In [10]:
# beyond node features, edge features are also common but we won't deal with that